In [1]:
import pandas as pd
import pdfplumber as pdf


In [2]:
doc = pdf.open('./src/src.pdf')
sents = [doc.pages[i].extract_text().split('\n') for i in range(len(doc.pages))]

doc_sents = []
for i in sents:
    doc_sents.extend(i)

doc_sents = [i.replace(',',' ').strip() for i in doc_sents]
doc_sents = pd.DataFrame(doc_sents)
doc_sents = doc_sents[doc_sents[0] != '']

In [3]:
doc_sents[0] = doc_sents[0].str.replace('[^0-9가-힣.()]',' ').str.strip()
doc_sents[0] = doc_sents[0].str.replace('[0-9]','#').str.strip()

/var/folders/4l/84htqxwn14n0jr_xvdc4tpbr0000gn/T/ipykernel_82793/827029015.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  doc_sents[0] = doc_sents[0].str.replace('[^0-9가-힣.()]',' ').str.strip()
/var/folders/4l/84htqxwn14n0jr_xvdc4tpbr0000gn/T/ipykernel_82793/827029015.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  doc_sents[0] = doc_sents[0].str.replace('[0-9]','#').str.strip()


In [4]:
doc_sents

,0
0,국민 체크카드회원 약관
1,제#조(회원)
2,회원이란 이 약관을 승인하고 국민카드(이하 카드사 라 함)에 국민체크카...
3,카드 라 함)발급을 신청하여 카드사로부터 카드를 발급 받은 분을 말합니다.
4,제#조(카드의 관리)
...,...
113,으로 본다는 내용을 명시하여 통지합니다. 회원이 변경예정일까지 이의를 제기하지 않았을
114,때에는 변경된 약관을 승인한 것으로 간주합니다.
116,제##조(본 약관에서 정하지 아니한 사항)
117,본 약관에서 정하지 아니한 사항과 본 약관의 해석에 관하여는 신용카드 개인회원 표준약관


In [5]:
# 문장복원
target = list(doc_sents[0])
doc_re_sents = []
flag = 0

for i in range(len(target)):
    if i <= flag:
        continue
    
    tp = target[i]
    if target[i][:2] == '제#':
        doc_re_sents.append(tp)
        continue
    
    for j in range(i+1,len(target)):
        if target[j][-2:] == '다.':
            flag = j
            tp += target[j]
            doc_re_sents.append(tp)
            break
        else:
            tp += target[j]
                

            

In [6]:
doc_re_sents

['제#조(회원)',
 '회원이란 이 약관을 승인하고    국민카드(이하  카드사 라 함)에    국민체크카드 (이하카드 라 함)발급을 신청하여 카드사로부터 카드를 발급 받은 분을 말합니다.',
 '제#조(카드의 관리)',
 '회원은 카드를 발급받는 즉시 카드서명란에 본인이 직접 서명하여야 하며  본인 이외의배우자  가족 등 다른 사람이 카드를 이용하게 하여서는 아니됩니다.',
 '카드의 소유권은 카드사에 있으므로 회원은 카드를 제 # 자에게 대여  양도 또는 담보의목적으로  이용할  수  없으며  선량한  관리자로서의  주의를  다하여  카드를  이용 관리하여야합니다.',
 '회원은 유효기한이 지난 카드 및 갱신 대체 재발급으로 인한 구카드를 이용할 수 없고이를 즉시 카드사에 반환하거나 이용이 불가능하도록 절단하여 분리해서 폐기하여야 합니다.',
 '제#항 내지 제#항의 이행을 태만히 하여 발생하는 책임은 회원에게 있습니다. 다만  회',
 '원에게 고의 또는 과실이 없는 경우에는 그러하지 아니합니다.제#조(제수수료 및 보증금 등 징구)카드사는 회원으로부터 카드와 관련하여 발급수수료 및 연회비 등을 징구할 수 있습니다.',
 '카드사는  무선인식  비접촉식  기능(이하      기능 이라  함)이  있는  카드에  대해서는회원으로부터  체크카드  보증금(이하   보증금   이라  함)을  징구할  수  있으며   보증금은회원의 카드 이용대금과 상계할 수 있습니다.',
 '제#조(비밀번호 및 면책)',
 '회원은  카드사가  정하는  방법에  따라  비밀번호를  신고하여야  하며   자동화기기를이용하거나   카드사와  비밀번호를  사용하기로  별도의  약정을  체결한  가맹점에서  카드를이용하는 경우 매 거래시 마다 카드사에 신고한 비밀번호를 직접 입력하여야 합니다.',
 '카드사는  비밀번호를  이용하는  거래시  회원이  조작한  비밀번호와  회원이  신고한비밀번호를 대조하여 일치함이 인정될 경우에  한하여 거래를 허용하며  일치함이 인정되어발생한  거래에

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1,3)).fit(doc_re_sents)
feature_vect_simple = tfidf.fit_transform(doc_re_sents) # 문서 벡터화 

In [8]:
tfidf.vocabulary_ # split 방식처럼 보인다

{'회원': 1877,
 '회원이란': 1948,
 '약관을': 949,
 '승인하고': 865,
 '국민카드': 272,
 '이하': 1193,
 '카드사': 1588,
 '국민체크카드': 269,
 '이하카드': 1200,
 '발급을': 598,
 '신청하여': 898,
 '카드사로부터': 1620,
 '카드를': 1567,
 '발급': 582,
 '받은': 577,
 '분을': 717,
 '말합니다': 527,
 '회원이란 약관을': 1949,
 '약관을 승인하고': 954,
 '승인하고 국민카드': 866,
 '국민카드 이하': 273,
 '이하 카드사': 1198,
 '카드사 국민체크카드': 1589,
 '국민체크카드 이하카드': 270,
 '이하카드 발급을': 1201,
 '발급을 신청하여': 599,
 '신청하여 카드사로부터': 899,
 '카드사로부터 카드를': 1621,
 '카드를 발급': 1568,
 '발급 받은': 585,
 '받은 분을': 578,
 '분을 말합니다': 718,
 '회원이란 약관을 승인하고': 1950,
 '약관을 승인하고 국민카드': 955,
 '승인하고 국민카드 이하': 867,
 '국민카드 이하 카드사': 274,
 '이하 카드사 국민체크카드': 1199,
 '카드사 국민체크카드 이하카드': 1590,
 '국민체크카드 이하카드 발급을': 271,
 '이하카드 발급을 신청하여': 1202,
 '발급을 신청하여 카드사로부터': 600,
 '신청하여 카드사로부터 카드를': 900,
 '카드사로부터 카드를 발급': 1622,
 '카드를 발급 받은': 1570,
 '발급 받은 분을': 586,
 '받은 분을 말합니다': 579,
 '카드의': 1649,
 '관리': 219,
 '카드의 관리': 1650,
 '회원은': 1899,
 '발급받는': 592,
 '즉시': 1433,
 '카드서명란에': 1640,
 '본인이': 696,
 '직접': 1476,
 '서명하여야': 822,
 '하며': 1731,
 '본인': 693,
 '

In [9]:
print(tfidf.transform(['잔액 부족 입력횟수 초과']).toarray()) # Input

[[0. 0. 0. ... 0. 0. 0.]]


In [10]:
print([i.toarray() for i in feature_vect_simple]) # sentence vector

[array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0.11582669, 0.11582669, 0.11582669, ..., 0.        , 0.        ,
        0.        ]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0.        , 0.        , 0.        , ..., 0.18304428, 0.18304428,
        0.18304428]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.]]), array([

In [15]:
[i.toarray() for i in feature_vect_simple][0].shape

(1, 1958)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
sim = []
# input text - document similarity
for i in range(len(doc_re_sents)):
    sim.append(cosine_similarity(tfidf.transform(['잔액 부족 입력횟수 초과']).toarray(), [i.toarray() for i in feature_vect_simple][i]))
sim

[array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.21091277]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.05226268]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]])]

In [12]:
doc_re_sents[13]

'#. 비밀번호의 입력횟수 초과#. 인출(이체)한도 또는 직불이용한도 초과#. 사고신고계좌#. 법적제한계좌#. 지급가능잔액(대출약정 계좌는 대출미사용 잔액 포함) 부족#. 당일잔액에 대한 잔액증명이 발급된 계좌교통기능 이용 등 일부 신용한도가 운영 되는 카드는 제 # 항 각호의 사유 외에 연체(카드이용대금   카드론  대금  또는  타  금융기관에  대한  채무)  및  파산   개인회생  신청  등회원의 신용상태가 현저히 악화된 경우에도 카드이용이 제한 또는 정지 될 수 있습니다.'

In [13]:
doc_re_sents[17]

'온라인 마감 결산을 위한 일시 전산가동 중단 시 및 예금 잔액 확인을 즉시 할 수 없어매출승인이 불가능한 무승인 가맹점에서는 카드이용이 제한될 수 있습니다.'

핵심은 완전한 문장을 살리는것

pdf 에서 정확하게 문장을 살려내는것.